In [1]:
import pandas as pd
import cobra
import cameo

In [2]:
wd = '/Users/alfredo/Documents/Jupyter/3_Isopropanol/ipynb/'

In [3]:
model = cameo.load_model('iML1515')

In [13]:
reacts_exchange = []
for r in model.reactions:
    reactants_p = [m for m in r.metabolites if r.metabolites[m]<0 and m.id.endswith('_p')]
    products_p = [m for m in r.metabolites if r.metabolites[m]>0 and m.id.endswith('_p')]
    reactants_c = [m for m in r.metabolites if r.metabolites[m]<0 and m.id.endswith('_c')]
    products_c = [m for m in r.metabolites if r.metabolites[m]>0 and m.id.endswith('_c')]
    
    if (len(reactants_p)>0 and len(reactants_c)>0) or (len(reactants_c)>0 and len(reactants_p)>0):
        reacts_exchange.append(r)

In [14]:
len(reacts_exchange)

205

In [41]:
def get_metabolite_transported(r):
    reactants = [m for m in r.metabolites if r.metabolites[m]<0]
    products = [m for m in r.metabolites if r.metabolites[m]>0]
    for react in reactants:
        if react.id.endswith('_c') and react.id.replace('_c','_p') in [p.id for p in products]:
            if react.id.replace('_c','') != 'h':
                return react
        elif react.id.endswith('_p') and react.id.replace('_p','_c') in [p.id for p in products]:
            if react.id.replace('_p','') != 'h':
                return model.metabolites.get_by_id(react.id.replace('_p','_c'))
    return None

In [59]:
rex_ok = [r for r in model.reactions if get_metabolite_transported(r) is not None and len([g for g in r.genes if g.id.startswith('b')])>0]
print(len(rex_ok))

365


In [102]:
df_rex = pd.DataFrame(index=[r.id for r in rex_ok])
df_rex['metabolite'] = [get_metabolite_transported(r).id.replace('_c','') for r in rex_ok]
df_rex['m_name'] = [get_metabolite_transported(r).name for r in rex_ok]
df_rex['reaction'] = [r.build_reaction_string() for r in rex_ok]
df_rex['genes'] = [' '.join(list([g.id for g in r.genes])) for r in rex_ok]
df_rex['description'] = [r.name for r in rex_ok]


In [103]:
df_rex

,metabolite,m_name,reaction,genes,description
SUCCt2_2pp,succ,Succinate,2.0 h_p + succ_p --> 2.0 h_c + succ_c,b3528 b0010,Succinate transport via proton symport (2 H) (...
SUCFUMtpp,fum,Fumarate,fum_p + succ_c <=> fum_c + succ_p,b4138 b0621 b4123,Succinate:fumarate antiporter (periplasm)
SULabcpp,so4,Sulfate,atp_c + h2o_c + so4_p --> adp_c + h_c + pi_c +...,b2423 b0765 b2424 b2422 b2425 b0763 b3917 b0764,Sulfate transport via ABC system (periplasm)
TSULabcpp,tsul,Thiosulfate,atp_c + h2o_c + tsul_p --> adp_c + h_c + pi_c ...,b2423 b2422 b2425 b3917 b2424,Thiosulfate transport via ABC system (periplasm)
TARTRt7pp,tartr__L,L-tartrate,succ_c + tartr__L_p <=> succ_p + tartr__L_c,b3063,Tartrate/succinate antiporter (periplasm)
TAURabcpp,taur,Taurine C2H7NO3S,atp_c + h2o_c + taur_p --> adp_c + h_c + pi_c ...,b0366 b0367 b0365,Taurine transport via ABC system (periplasm)
THMDt2pp_copy1,thymd,Thymidine C10H14N2O5,h_p + thymd_p --> h_c + thymd_c,b2964 b2393,Thymidine transport in via proton symport (per...
Kt2pp,k,Potassium,h_p + k_p --> h_c + k_c,b1363 b3849 b1250 b3747 b3290 b1291,Potassium transport in via proton symport (per...
AKGt2rpp,akg,2-Oxoglutarate,akg_p + h_p <=> akg_c + h_c,b2587,2-oxoglutarate reversible transport via sympor...
XYLt2pp,xyl__D,D-Xylose,h_p + xyl__D_p --> h_c + xyl__D_c,b4031,D-xylose transport in via proton symport (peri...


In [114]:
genes_mets = pd.Series()
for r_id in df_rex.index:
    genes = df_rex.loc[r_id,'genes'].split()
    for g_id in genes:
        if g_id in genes_mets.index:
            genes_mets[g_id] += ' '+df_rex.loc[r_id,'m_name']
        else:
            genes_mets[g_id] = df_rex.loc[r_id,'m_name']
len(genes_mets)

340

In [116]:
genes_mets.to_excel('/Users/alfredo/Desktop/exchange_genes_metabolites.xlsx')

In [121]:
df_rex[df_rex['metabolite']=='but']

,metabolite,m_name,reaction,genes,description
BUTt2rpp,but,Butyrate (n-C4:0),but_p + h_p <=> but_c + h_c,b2223,"Butyrate transport via proton symport, reversi..."


In [122]:
model.reactions.BUTt2rpp

Reaction identifier,BUTt2rpp
Name,"Butyrate transport via proton symport, reversible (periplasm)"
Memory address,0x0122c5dcc0
Stoichiometry,but_p + h_p <=> but_c + h_c Butyrate (n-C4:0) + H+ <=> Butyrate (n-C4:0) + H+
GPR,b2223
Lower bound,-1000.0
Upper bound,1000.0
